In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/term_paper/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# smplx
need_smplx=False
try:
    import smplx
except ModuleNotFoundError:
    need_smplx=True

if need_smplx:
    !pip install smplx
    !git clone https://github.com/vchoutas/smplx
    %cd smplx
    !python setup.py install
    %cd ..


# detectron2
need_detectron=False
try:
    import detectron2
except ModuleNotFoundError:
    need_detectron=True   

if need_detectron:
    !git clone https://github.com/facebookresearch/detectron2.git detectron2_repo
    !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

In [ ]:
import importlib
import neural_rendering
import utils.smpl_to_smplx
import utils.smplx_to_disps
import utils.renderers
import utils.inpainting
importlib.reload(neural_rendering)
importlib.reload(utils.smpl_to_smplx)
importlib.reload(utils.smplx_to_disps)
importlib.reload(utils.renderers)
importlib.reload(utils.inpainting)

In [ ]:
import smplx
import matplotlib.pyplot as plt
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.ops import SubdivideMeshes
from pytorch3d.renderer import (
    PerspectiveCameras,
    TexturesUV,
    TexturesVertex
)

from utils.plot_structures import plot_structure
from utils.smplx_to_disps import smplx2disps
from utils.inpainting import get_disps_inpaint
from utils.camera_calibration import get_camera_parameters
from utils.renderers import get_renderers
from utils.pointrend_segmentation import get_pointrend_segmentation

from neural_rendering import neural_renderer, construct_textured_mesh

In [ ]:
### Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
subject = 1
pose = '00000025' # T-pose
subdivision = False

obj_path = drive_path + 'smplx/smplx_uv.obj'
subd_obj_path = drive_path + 'smplx/subd_smplx_uv.obj'
uv_mask_img = drive_path + 'smplx/smplx_uv.png'

save_path = drive_path + 'humbi_smplx'

smplx_model_path = drive_path + 'smplx'
smplx_model = smplx.SMPLXLayer(smplx_model_path, 'neutral').to(device)

In [ ]:
if not subdivision:
    geometry, texture = neural_renderer(smplx_model, subject, pose, 1, obj_path, subdivision, rescale_factor=8, save_path=save_path)
else:
    geometry, texture = neural_renderer(smplx_model, subject, pose, 30, subd_obj_path, subdivision, rescale_factor=2, save_path=save_path)

In [ ]:
# Extract geometry
global_orient, transl, body_pose, left_hand_pose, right_hand_pose, jaw_pose, expression, betas, scale, verts_disps = geometry

# Store geometry into displacements along normals + get displaced and initial mesh
learned_geometry, init_mesh, displaced_mesh = smplx2disps(smplx_model, betas, scale, verts_disps, subdivision, smoothing=2)

# Construct displacement map by interpolating values between uv vertex coordinates (inpainting) : for now only available for no subdivision!
if not subdivision:
    displacement_map, mask = get_disps_inpaint(subject, learned_geometry, obj_path, uv_mask_img, mask_disps=True)

plot_structure([init_mesh, displaced_mesh])

In [ ]:
if not subdivision:
    plt.figure(figsize=(10, 10))
    plt.imshow(displacement_map.cpu(), cmap='gray')
    plt.axis("off")

    plt.figure(figsize=(10, 10))
    plt.imshow(mask.cpu(), cmap='gray')
    plt.axis("off")

plt.figure(figsize=(10, 10))
plt.imshow(texture[0].cpu())
plt.axis("off")

In [ ]:
cam_idx = 72
img_size = (1080, 1920)
render_res = (int(1080/3), int(1920/3))

# Extract camera parameters and construct camera
R, T, f, p = get_camera_parameters(subject, cam_idx)
cameras = PerspectiveCameras(focal_length=-f, principal_point=p, R=R, T=T, in_ndc=False, image_size=(img_size,), device=device)

# Segment person in photo from camera viewpoint
photo_path = 'subject_%s/body/%s/image/image%s.jpg' % (subject, pose, str(cam_idx).zfill(7))
photo, silh_photo, rgb_photo = get_pointrend_segmentation(photo_path, device=device)

photo = photo[::3, ::3].to(device)
silh_photo = silh_photo[0, ::3, ::3].float().to(device)
rgb_photo = rgb_photo[0, ::3, ::3].to(device)

# Construct mesh
obj_mesh = load_obj(obj_path, load_textures=False)
faces_uvs = obj_mesh[1].textures_idx.unsqueeze(0).to(device)
verts_uvs = obj_mesh[2].verts_uvs.unsqueeze(0).to(device)
texture_uv = TexturesUV(maps=texture, faces_uvs=faces_uvs, verts_uvs=verts_uvs)
mesh = construct_textured_mesh(smplx_model, texture_uv, global_orient, transl, body_pose, left_hand_pose, right_hand_pose, jaw_pose, expression, betas, scale, subdivision, verts_disps)

# Render mesh from camera viewpoint
silhouette_renderer, phong_renderer = get_renderers(cameras, render_res, device=device)
phong_render = phong_renderer(mesh).detach()
silhouette_render = silhouette_renderer(mesh).detach()

rgb_render = phong_render[0, ..., :3]
silh_render = silhouette_render[0, ..., 3]

In [ ]:
# Target Image
plt.figure(figsize=(16, 9))
plt.imshow(photo.cpu())
plt.axis("off")

# RGB Render
plt.figure(figsize=(16, 9))
plt.imshow(rgb_render.cpu())
plt.axis("off")

# RGB Photo
plt.figure(figsize=(16, 9))
plt.imshow(rgb_photo.cpu())
plt.axis("off")

# Mask Render
plt.figure(figsize=(16, 9))
plt.imshow(silh_render.cpu())
plt.axis("off")

# Mask Photo
plt.figure(figsize=(16, 9))
plt.imshow(silh_photo.cpu())
plt.axis("off")

In [ ]:
import cv2
import numpy as np
from PIL import Image

# Save rgb color map as texture
nrm_rgb_map = (texture[0].cpu().numpy() * 255.0).astype(np.uint8)
Image.fromarray(nrm_rgb_map).save('test.png')

# Save displacement map as texture
nrm_disps_map = cv2.normalize(displacement_map.numpy(), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
Image.fromarray(nrm_disps_map, mode='F').save('test.tiff')

# Save geometry in npz file
outfile = 'output_subject_%d.npz' % subject
np.savez(outfile, body_pose=body_pose.numpy(), betas=betas.numpy(), scale=scale.numpy(), verts_disps=verts_disps.numpy(), learned_geometry=learned_geometry.numpy())

# Loading npz file
npzfile = np.load(outfile)
npzfile.files # see all stored arrays in npz file
npzfile['body_pose'] # call body_pose array stored into npz file